## The following section is for Colab Users.
### Just run the following code cells

In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://bitbucket.org/habedi/datasets/raw/b6769c4664e7ff68b001e2f43bc517888cbe3642/spark/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!rm -rf spark-3.0.2-bin-hadoop2.7.tgz*
!pip -q install findspark pyspark graphframes

     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 199 kB 53.5 MB/s 
     |████████████████████████████████| 154 kB 61.3 MB/s 


In [2]:
!wget https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.0-s_2.12/graphframes-0.8.2-spark3.0-s_2.12.jar -P /content/spark-3.0.2-bin-hadoop2.7/jars/
!cp /content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar /content/spark-3.0.2-bin-hadoop2.7/graphframes-0.8.2-spark3.0-s_2.12.zip

--2022-07-14 16:20:53--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.0-s_2.12/graphframes-0.8.2-spark3.0-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 13.224.2.102, 13.224.2.91, 13.224.2.85, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|13.224.2.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 247882 (242K) [binary/octet-stream]
Saving to: ‘/content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar’

graphframes-0.8.2-s 100%[===================>] 242.07K  --.-KB/s    in 0.02s   

2022-07-14 16:20:53 (10.5 MB/s) - ‘/content/spark-3.0.2-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar’ saved [247882/247882]



In [3]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = os.environ["SPARK_HOME"]

os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

In [4]:
import findspark
findspark.init()

In [5]:
!export PYSPARK_SUBMIT_ARGS="--master local[*] pyspark-shell"
!export PYSPARK_DRIVER_PYTHON=jupyter
!export PYSPARK_DRIVER_PYTHON_OPTS=notebook

In [6]:
from pyspark.sql import SparkSession
from graphframes import *

spark = SparkSession.builder.master("local[*]").appName("GraphFrames").getOrCreate()

In [7]:
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell"

**************************************************************************
**************************************************************************
**************************************************************************

In [8]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Read departuredelays.csv in Edge DataFrame
### Read airport-codes-na.txt in Vertix DataFrame (the separator is Tab i.e sep = '\t' )

In [37]:
vertex = spark.read.csv('/content/airport-codes-na.txt',sep='\t',header=True)
edges = spark.read.csv('/content/departuredelays.csv',header=True,inferSchema=True)

#### The US flight delays data set has five columns:
- The <b>date</b> column contains an integer like 02190925 . When converted, this maps to 02-19 09:25 am.
- The <b>delay</b> column gives the delay in minutes between the scheduled and actual departure times. Early departures show negative numbers.
- The <b>distance</b> column gives the distance in miles from the origin airport to the destination airport.
- The <b>origin</b> column contains the origin IATA airport code.
- The <b>destination</b> column contains the destination IATA airport code.

#### The airport-codes data set has four columns:
- The <b>IATA</b> column contains IATA airport code.
- The <b>City, State, and Country</b> columns contains information about the airport location. 

In [38]:
vertex.show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
|   Alliance|   NE|    USA| AIA|
|     Alpena|   MI|    USA| APN|
|    Altoona|   PA|    USA| AOO|
|   Amarillo|   TX|    USA| AMA|
|Anahim Lake|   BC| Canada| YAA|
|  Anchorage|   AK|    USA| ANC|
|   Appleton|   WI|    USA| ATW|
|     Arviat|  NWT| Canada| YEK|
|  Asheville|   NC|    USA| AVL|
|      Aspen|   CO|    USA| ASE|
+-----------+-----+-------+----+
only showing top 20 rows



In [39]:
edges.show()

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
|1061215|   -6|     602|   ABE|        ATL|
|1061725|   69|     602|   ABE|        ATL|
|1061230|    0|     369|   ABE|        DTW|
|1060625|   -3|     602|   ABE|        ATL|
|1070600|    0|     369|   ABE|        DTW|
|1071725|    0|     602|   ABE|        ATL|
|1071230|    0|     369|   ABE|        DTW|
|1070625|    0|     602|   ABE|        ATL|
|1071219|    0|     569|   ABE|        ORD|
|1080600|    0|     369|   ABE| 

### In the vertix DataFrame, drop any duplicated rows with the same  IATA code.

In [40]:
vertex.count()

526

In [42]:
vertex = vertex.drop_duplicates(subset=['IATA'])
vertex.count()

524

### In the edges DataFrame:
- Rename the <b>date</b> columns to become <b>tripid</b>.
- Rename the <b>origin</b> columns to become <b>src</b>.
- Rename the <b>destination</b> columns to become <b>dst</b>.

In [46]:
edges = edges.withColumnRenamed('date','tripid').withColumnRenamed('origin','src').withColumnRenamed('destination','dst')
edges.show()

+-------+-----+--------+---+---+
| tripid|delay|distance|src|dst|
+-------+-----+--------+---+---+
|1011245|    6|     602|ABE|ATL|
|1020600|   -8|     369|ABE|DTW|
|1021245|   -2|     602|ABE|ATL|
|1020605|   -4|     602|ABE|ATL|
|1031245|   -4|     602|ABE|ATL|
|1030605|    0|     602|ABE|ATL|
|1041243|   10|     602|ABE|ATL|
|1040605|   28|     602|ABE|ATL|
|1051245|   88|     602|ABE|ATL|
|1050605|    9|     602|ABE|ATL|
|1061215|   -6|     602|ABE|ATL|
|1061725|   69|     602|ABE|ATL|
|1061230|    0|     369|ABE|DTW|
|1060625|   -3|     602|ABE|ATL|
|1070600|    0|     369|ABE|DTW|
|1071725|    0|     602|ABE|ATL|
|1071230|    0|     369|ABE|DTW|
|1070625|    0|     602|ABE|ATL|
|1071219|    0|     569|ABE|ORD|
|1080600|    0|     369|ABE|DTW|
+-------+-----+--------+---+---+
only showing top 20 rows



### In the Vertix DataFrame:
- Rename the <b>IATA</b> columns to become <b>id</b>.

In [47]:
vertex = vertex.withColumnRenamed('IATA','id')
vertex.show()

+-------------------+-----+-------+---+
|               City|State|Country| id|
+-------------------+-----+-------+---+
|         Binghamton|   NY|    USA|BGM|
|            Lebanon|   NH|    USA|LEB|
|           Montreal|   PQ| Canada|YUL|
|         Dillingham|   AK|    USA|DLG|
|International Falls|   MN|    USA|INL|
|         Wolf Point|   MT|    USA|OLF|
|        New Orleans|   LA|    USA|MSY|
|            Toronto|   ON| Canada|YTO|
|            Spokane|   WA|    USA|GEG|
|              Havre|   MT|    USA|HVR|
|            Burbank|   CA|    USA|BUR|
|      Orange County|   CA|    USA|SNA|
|             Dryden|   ON| Canada|YHD|
|         Fort Dodge|   IA|    USA|FOD|
|          Green Bay|   WI|    USA|GRB|
|        Great Falls|   MT|    USA|GTF|
|              Homer|   AK|    USA|HOM|
|        Idaho Falls|   ID|    USA|IDA|
|      Sioux Lookout|   ON| Canada|YXL|
|       Grand Rapids|   MI|    USA|GRR|
+-------------------+-----+-------+---+
only showing top 20 rows



### Create GraphFrame from Vertix and Edges DataFrames

In [48]:
g = GraphFrame(vertex,edges)

In [111]:
g.edges.show()

+-------+-----+--------+---+---+
| tripid|delay|distance|src|dst|
+-------+-----+--------+---+---+
|1011245|    6|     602|ABE|ATL|
|1020600|   -8|     369|ABE|DTW|
|1021245|   -2|     602|ABE|ATL|
|1020605|   -4|     602|ABE|ATL|
|1031245|   -4|     602|ABE|ATL|
|1030605|    0|     602|ABE|ATL|
|1041243|   10|     602|ABE|ATL|
|1040605|   28|     602|ABE|ATL|
|1051245|   88|     602|ABE|ATL|
|1050605|    9|     602|ABE|ATL|
|1061215|   -6|     602|ABE|ATL|
|1061725|   69|     602|ABE|ATL|
|1061230|    0|     369|ABE|DTW|
|1060625|   -3|     602|ABE|ATL|
|1070600|    0|     369|ABE|DTW|
|1071725|    0|     602|ABE|ATL|
|1071230|    0|     369|ABE|DTW|
|1070625|    0|     602|ABE|ATL|
|1071219|    0|     569|ABE|ORD|
|1080600|    0|     369|ABE|DTW|
+-------+-----+--------+---+---+
only showing top 20 rows



### Determine the number of airports

In [49]:
from pyspark.sql.functions import *

In [50]:
g.vertices.select(count('id').alias('Number Of Airports')).show()

+------------------+
|Number Of Airports|
+------------------+
|               524|
+------------------+



### Determine the number of trips 

In [51]:
g.edges.count()

1391578

In [52]:
g.edges.select(count('*').alias('Number Of Trips')).show()

+---------------+
|Number Of Trips|
+---------------+
|        1391578|
+---------------+



### What is the longest delay?

In [56]:
edgs = g.edges

In [61]:
edgs.select(max('delay').alias('Maximum Delay')).show()

+-------------+
|Maximum Delay|
+-------------+
|         1642|
+-------------+



### Find out the number of delayed flights vs. early flights (flights that departed before actual time)

In [65]:
g.edges.filter('delay > 0').count()

591727

In [62]:
edgs.filter('delay > 0 ').count()

591727

In [63]:
edgs.filter('delay < 0').count()

668729

### What flight destinations departing SFO are most likely to have significant delays? Select the top 10
#### Hint: you should get the average delay for each destination for trips that depart from SFO only

In [100]:
sfo = g.edges.filter('src = "SFO"').groupBy(['src','dst']).avg('delay')
sfo.show()

+---+---+------------------+
|src|dst|        avg(delay)|
+---+---+------------------+
|SFO|TUS|14.923913043478262|
|SFO|BOI| 16.21123595505618|
|SFO|MSY|           14.2125|
|SFO|STL|         20.203125|
|SFO|SMF|11.426984126984127|
|SFO|MRY| 8.306818181818182|
|SFO|EUG|13.691449814126393|
|SFO|PIT|17.345454545454544|
|SFO|MCI|18.254054054054055|
|SFO|ASE|19.846153846153847|
|SFO|RDD|11.555555555555555|
|SFO|ANC|              17.0|
|SFO|AUS| 13.96763202725724|
|SFO|ORD|15.339675433687745|
|SFO|CLT| 4.185779816513762|
|SFO|MKE|  9.24390243902439|
|SFO|LMT|5.0777777777777775|
|SFO|DFW| 9.971256684491978|
|SFO|SBP|13.530066815144766|
|SFO|MCO|12.372972972972972|
+---+---+------------------+
only showing top 20 rows



In [99]:
sfo.sort(desc('avg(delay)')).show(10)

+---+---+------------------+
|src|dst|        avg(delay)|
+---+---+------------------+
|SFO|JAC| 30.78846153846154|
|SFO|OKC|24.822222222222223|
|SFO|SUN|22.696629213483146|
|SFO|COS| 22.58888888888889|
|SFO|SAT|             22.16|
|SFO|STL|         20.203125|
|SFO|HNL|19.982608695652175|
|SFO|ASE|19.846153846153847|
|SFO|CEC|19.089820359281436|
|SFO|MDW|18.771929824561404|
+---+---+------------------+
only showing top 10 rows



### Find the Incoming connections to the airport sorted in Desc. order.

In [103]:
g.inDegrees.sort(desc('inDegree')).show()

+---+--------+
| id|inDegree|
+---+--------+
|ATL|   90434|
|DFW|   66050|
|ORD|   61967|
|LAX|   53601|
|DEN|   50921|
|IAH|   42700|
|PHX|   39721|
|SFO|   38988|
|LAS|   32994|
|CLT|   28388|
|MCO|   27959|
|EWR|   27652|
|LGA|   25469|
|BOS|   25360|
|SLC|   25323|
|JFK|   23484|
|DTW|   23310|
|SEA|   23074|
|MSP|   22385|
|MIA|   21805|
+---+--------+
only showing top 20 rows



### Find the Outgoing connections from the airport sorted in Desc. order.

In [104]:
g.outDegrees.sort(desc('outDegree')).show()

+---+---------+
| id|outDegree|
+---+---------+
|ATL|    91484|
|DFW|    68482|
|ORD|    64228|
|LAX|    54086|
|DEN|    53148|
|IAH|    43361|
|PHX|    40155|
|SFO|    39483|
|LAS|    33107|
|CLT|    28402|
|MCO|    28313|
|EWR|    27656|
|SLC|    25868|
|LGA|    25458|
|BOS|    25348|
|MSP|    24031|
|JFK|    23572|
|DTW|    23421|
|SEA|    23078|
|MIA|    21817|
+---+---------+
only showing top 20 rows



### Use motif finding to answer this question: which delays could we blame on SFO?
#### Hint: this practically means that SFO is a transit station

In [105]:
triangle = g.find('(v1)-[e1]->(v2);(v2)-[e2]->(v3)')
triangle.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  v1|                  e1|                  v2|                  e2|                  v3|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Atlanta, GA, USA...|[1012110, -3, 369...|[New Orleans, LA,...|[1011335, -4, 389...|[Dallas, TX, USA,...|
|[Atlanta, GA, USA...|[1012110, -3, 369...|[New Orleans, LA,...|[1011550, -2, 389...|[Dallas, TX, USA,...|
|[Atlanta, GA, USA...|[1012110, -3, 369...|[New Orleans, LA,...|[1011845, -12, 58...|[Miami, FL, USA, ...|
|[Atlanta, GA, USA...|[1012110, -3, 369...|[New Orleans, LA,...|[1010825, -1, 389...|[Dallas, TX, USA,...|
|[Atlanta, GA, USA...|[1012110, -3, 369...|[New Orleans, LA,...|[1011505, -4, 586...|[Miami, FL, USA, ...|
|[Atlanta, GA, USA...|[1012110, -3, 369...|[New Orleans, LA,...|[1010650, -7, 389...|[Dallas, TX, USA,...|
|[Atlanta, GA, USA...|[1012110, -3, 3

In [113]:
triangle.filter('v2.id = "SFO"').filter('e1.tripid < e2.tripid').show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  v1|                  e1|                  v2|                  e2|                  v3|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Albuquerque, NM,...|[1010600, -7, 779...|[San Francisco, C...|[1011250, 55, 224...|[New York, NY, US...|
|[Albuquerque, NM,...|[1010600, -7, 779...|[San Francisco, C...|[1012230, 0, 2247...|[New York, NY, US...|
|[Albuquerque, NM,...|[1010600, -7, 779...|[San Francisco, C...|[1010705, -7, 224...|[New York, NY, US...|
|[Albuquerque, NM,...|[1010600, -7, 779...|[San Francisco, C...|[1010620, -3, 224...|[Miami, FL, USA, ...|
|[Albuquerque, NM,...|[1010600, -7, 779...|[San Francisco, C...|[1010915, -3, 293...|[Los Angeles, CA,...|
|[Albuquerque, NM,...|[1010600, -7, 779...|[San Francisco, C...|[1011005, -8, 127...|[Dallas, TX, USA,...|
|[Albuquerque, NM,...|[1010600, -7, 7

### Determine Airport Ranking in Desc. order using PageRank algorithm

In [115]:
pr = g.pageRank(maxIter= 3)

In [119]:
pr.vertices.sort(desc('pagerank')).show()

+--------------+-----+-------+---+------------------+
|          City|State|Country| id|          pagerank|
+--------------+-----+-------+---+------------------+
|       Atlanta|   GA|    USA|ATL|35.605263744382604|
|        Dallas|   TX|    USA|DFW|25.647472133009476|
|       Chicago|   IL|    USA|ORD| 24.40566620587968|
|        Denver|   CO|    USA|DEN|17.241685512286086|
|       Houston|   TX|    USA|IAH|  14.4925835065425|
|   Los Angeles|   CA|    USA|LAX|13.812051256434101|
| San Francisco|   CA|    USA|SFO|11.352607546983776|
|       Phoenix|   AZ|    USA|PHX| 10.52548731539901|
|Salt Lake City|   UT|    USA|SLC|10.506004098870957|
|     Las Vegas|   NV|    USA|LAS| 8.408451915634176|
|       Seattle|   WA|    USA|SEA| 7.530294879937547|
|        Newark|   NJ|    USA|EWR| 7.219522198674636|
|       Detroit|   MI|    USA|DTW| 7.083286560333701|
|       Orlando|   FL|    USA|MCO| 6.980986825250971|
|   Minneapolis|   MN|    USA|MSP| 6.925286688278695|
|     Charlotte|   NC|    US

In [117]:
pr.edges.show()

+-------+-----+--------+---+---+--------------------+
| tripid|delay|distance|src|dst|              weight|
+-------+-----+--------+---+---+--------------------+
|1011450|   -4|    1451|MSY|LAX|9.730466089325679E-5|
|1021445|    8|    1451|MSY|LAX|9.730466089325679E-5|
|1020700|   -1|    1451|MSY|LAX|9.730466089325679E-5|
|1021701|   15|    1451|MSY|LAX|9.730466089325679E-5|
|1031445|   22|    1451|MSY|LAX|9.730466089325679E-5|
|1030700|   -2|    1451|MSY|LAX|9.730466089325679E-5|
|1031701|   33|    1451|MSY|LAX|9.730466089325679E-5|
|1040700|   -4|    1451|MSY|LAX|9.730466089325679E-5|
|1041701|    7|    1451|MSY|LAX|9.730466089325679E-5|
|1051445|  480|    1451|MSY|LAX|9.730466089325679E-5|
|1051701|   30|    1451|MSY|LAX|9.730466089325679E-5|
|1050700|   -4|    1451|MSY|LAX|9.730466089325679E-5|
|1071523|    1|    1451|MSY|LAX|9.730466089325679E-5|
|1081523|    2|    1451|MSY|LAX|9.730466089325679E-5|
|1091523|    7|    1451|MSY|LAX|9.730466089325679E-5|
|1101523|   -2|    1451|MSY|

## Determine the most popular flights (single city hops)

In [120]:
flights = g.find('(v1)-[e1]->(v2)')

In [124]:
flights.groupBy(['v1','v2']).count().sort(desc('count')).show()

+--------------------+--------------------+-----+
|                  v1|                  v2|count|
+--------------------+--------------------+-----+
|[San Francisco, C...|[Los Angeles, CA,...| 3232|
|[Los Angeles, CA,...|[San Francisco, C...| 3198|
|[Las Vegas, NV, U...|[Los Angeles, CA,...| 3016|
|[Los Angeles, CA,...|[Las Vegas, NV, U...| 2964|
|[New York, NY, US...|[Los Angeles, CA,...| 2720|
|[Los Angeles, CA,...|[New York, NY, US...| 2719|
|[Atlanta, GA, USA...|[New York, NY, US...| 2501|
|[New York, NY, US...|[Atlanta, GA, USA...| 2500|
|[Los Angeles, CA,...|[Phoenix, AZ, USA...| 2394|
|[Phoenix, AZ, USA...|[Los Angeles, CA,...| 2387|
|[Honolulu, Oahu, ...|[Kahului, Maui, H...| 2380|
|[Kahului, Maui, H...|[Honolulu, Oahu, ...| 2379|
|[Los Angeles, CA,...|[San Diego, CA, U...| 2215|
|[San Diego, CA, U...|[Los Angeles, CA,...| 2214|
|[San Jose, CA, US...|[Los Angeles, CA,...| 2208|
|[Los Angeles, CA,...|[San Jose, CA, US...| 2201|
|[Atlanta, GA, USA...|[Orlando, FL, USA...| 2136|


### Find and Save a Subragph that obtained from the following pattern:
#### The flight starts from an airport and return back to the same airport through 2 other airports.

In [125]:
subGraph = g.find('(v1)-[e1]->(v2);(v2)-[e2]->(v3);(v3)-[e3]->(v1)')

In [126]:
subGraph.show()

KeyboardInterrupt: ignored

In [ ]:
subGraph.write.csv('subGraph')

In [ ]:
# subGraph.vertices.write.parquet("vertices")
# subGraph.edges.write.parquet("edges")